# PostgreSQL + pgvector ML Examples

This notebook demonstrates machine learning workflows using PostgreSQL with the pgvector extension for storing and querying vector embeddings alongside structured experiment data.

In [ ]:
// Import required packages and set up database connection
#r "nuget: Npgsql, 9.0.1"
#r "nuget: Pgvector, 0.2.0"
#r "nuget: System.Text.Json, 9.0.0"
#r "nuget: Dapper, 2.1.35"

using Npgsql;
using Pgvector;
using System.Text.Json;
using System.Data;
using Dapper;

// Use configuration from our project
var connectionString = NotebookConfiguration.PostgreSQL.GetConnectionString();
Console.WriteLine($"Connection configured for: {connectionString.Split(';')[0]}");

## Database Schema Setup

Create the required tables for ML experiment tracking and vector storage.

In [ ]:
// Create database schema and tables
using var connection = new NpgsqlConnection(connectionString);
await connection.OpenAsync();

// Enable pgvector extension
await connection.ExecuteAsync("CREATE EXTENSION IF NOT EXISTS vector;");

// Create schema for ML experiments
await connection.ExecuteAsync("CREATE SCHEMA IF NOT EXISTS ml_schema;");

// Create experiments table
await connection.ExecuteAsync(@"
    CREATE TABLE IF NOT EXISTS ml_schema.experiments (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        model_type VARCHAR(100) NOT NULL,
        description TEXT,
        parameters JSONB,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );");

// Create embeddings table with vector column
await connection.ExecuteAsync(@"
    CREATE TABLE IF NOT EXISTS ml_schema.embeddings (
        id SERIAL PRIMARY KEY,
        experiment_id INTEGER REFERENCES ml_schema.experiments(id),
        text_content TEXT NOT NULL,
        embedding vector(384),
        metadata JSONB,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );");

// Create index for vector similarity search
await connection.ExecuteAsync(@"
    CREATE INDEX IF NOT EXISTS embeddings_vector_idx 
    ON ml_schema.embeddings USING hnsw (embedding vector_cosine_ops);");

Console.WriteLine("✅ Database schema created successfully!");

## Experiment Tracking

Track ML experiments with structured metadata and performance metrics.

In [ ]:
// Helper method to create ML experiments
async Task<int> CreateExperimentAsync(string name, string modelType, string description, object parameters)
{
    var parametersJson = JsonSerializer.Serialize(parameters);
    
    using var connection = new NpgsqlConnection(connectionString);
    
    var sql = @"
        INSERT INTO ml_schema.experiments (name, model_type, description, parameters)
        VALUES (@name, @modelType, @description, @parameters::jsonb)
        RETURNING id;";
    
    var experimentId = await connection.QuerySingleAsync<int>(sql, new 
    { 
        name, 
        modelType, 
        description, 
        parameters = parametersJson 
    });
    
    Console.WriteLine($"✅ Created experiment: {experimentId} - {name}");
    return experimentId;
}

// Create sample experiments
var sentimentExperiment = await CreateExperimentAsync(
    "Sentiment Analysis v1",
    "Classification", 
    "Binary sentiment classification using pre-trained embeddings",
    new { learning_rate = 0.001, batch_size = 32, epochs = 10 }
);

var similarityExperiment = await CreateExperimentAsync(
    "Document Similarity",
    "Embedding", 
    "Semantic document similarity using vector embeddings",
    new { model = "sentence-transformers/all-MiniLM-L6-v2", dimension = 384 }
);

## Vector Operations

Generate embeddings and store them for similarity search operations.

In [ ]:
// Simulate embedding generation (in practice, use ML.NET or call external API)
Vector GenerateEmbedding(string text)
{
    // This is a simplified example - use actual embedding models in production
    var random = new Random(text.GetHashCode());
    var values = Enumerable.Range(0, 384)
        .Select(_ => (float)(random.NextDouble() - 0.5))
        .ToArray();
    
    // Normalize the vector
    var magnitude = Math.Sqrt(values.Sum(x => x * x));
    for (int i = 0; i < values.Length; i++)
    {
        values[i] /= (float)magnitude;
    }
    
    return new Vector(values);
}

// Sample documents for embedding
var documents = new[]
{
    "Machine learning is transforming how we process data",
    "Artificial intelligence enables automated decision making", 
    "Deep learning networks can recognize complex patterns",
    "Natural language processing helps computers understand text",
    "Computer vision allows machines to interpret visual information",
    "Reinforcement learning teaches agents through trial and error"
};

// Store embeddings in database
using var connection = new NpgsqlConnection(connectionString);

foreach (var doc in documents)
{
    var embedding = GenerateEmbedding(doc);
    
    var sql = @"
        INSERT INTO ml_schema.embeddings (experiment_id, text_content, embedding, metadata)
        VALUES (@experimentId, @content, @embedding, @metadata::jsonb)";
    
    await connection.ExecuteAsync(sql, new
    {
        experimentId = similarityExperiment,
        content = doc,
        embedding,
        metadata = JsonSerializer.Serialize(new { 
            length = doc.Length, 
            word_count = doc.Split(' ').Length 
        })
    });
}

Console.WriteLine($"✅ Stored {documents.Length} document embeddings");

## Similarity Search

Perform vector similarity searches to find related documents.

In [ ]:
// Perform similarity search
async Task<IEnumerable<object>> FindSimilarDocumentsAsync(string queryText, int limit = 5)
{
    var queryEmbedding = GenerateEmbedding(queryText);
    
    using var connection = new NpgsqlConnection(connectionString);
    
    var sql = @"
        SELECT 
            id,
            text_content,
            1 - (embedding <=> @queryEmbedding) as similarity_score,
            metadata
        FROM ml_schema.embeddings
        WHERE experiment_id = @experimentId
        ORDER BY embedding <=> @queryEmbedding
        LIMIT @limit";
    
    return await connection.QueryAsync(sql, new
    {
        queryEmbedding,
        experimentId = similarityExperiment,
        limit
    });
}

// Test similarity search
var searchQuery = "AI systems for text analysis";
var similarDocs = await FindSimilarDocumentsAsync(searchQuery, 3);

Console.WriteLine($"🔍 Similar documents to: '{searchQuery}'\n");
foreach (var doc in similarDocs)
{
    Console.WriteLine($"Score: {doc.similarity_score:F3} | {doc.text_content}");
}

## Analytics & Reporting

Analyze experiment results and vector similarity patterns.

In [ ]:
// Get experiment statistics
using var connection = new NpgsqlConnection(connectionString);

var experimentStats = await connection.QueryAsync(@"
    SELECT 
        e.name,
        e.model_type,
        COUNT(em.id) as embedding_count,
        AVG(array_length(string_to_array(em.text_content, ' '), 1)) as avg_word_count,
        e.created_at
    FROM ml_schema.experiments e
    LEFT JOIN ml_schema.embeddings em ON e.id = em.experiment_id
    GROUP BY e.id, e.name, e.model_type, e.created_at
    ORDER BY e.created_at DESC");

Console.WriteLine("📊 Experiment Statistics:\n");
foreach (var stat in experimentStats)
{
    Console.WriteLine($"Experiment: {stat.name}");
    Console.WriteLine($"  Type: {stat.model_type}");
    Console.WriteLine($"  Embeddings: {stat.embedding_count}");
    Console.WriteLine($"  Avg Words: {stat.avg_word_count:F1}");
    Console.WriteLine($"  Created: {stat.created_at:yyyy-MM-dd HH:mm}");
    Console.WriteLine();
}

## Vector Clustering Analysis

Analyze similarity patterns in the vector space.

In [ ]:
// Calculate similarity matrix between all documents
var allEmbeddings = await connection.QueryAsync(@"
    SELECT id, text_content, embedding
    FROM ml_schema.embeddings
    WHERE experiment_id = @experimentId
    ORDER BY id", 
    new { experimentId = similarityExperiment });

Console.WriteLine("🔢 Document Similarity Matrix:\n");

var embedList = allEmbeddings.ToList();
Console.Write("".PadRight(4));
for (int i = 0; i < embedList.Count; i++)
{
    Console.Write($"Doc{i + 1}".PadRight(6));
}
Console.WriteLine();

for (int i = 0; i < embedList.Count; i++)
{
    Console.Write($"Doc{i + 1}".PadRight(4));
    
    for (int j = 0; j < embedList.Count; j++)
    {
        if (i == j)
        {
            Console.Write("1.000".PadRight(6));
        }
        else
        {
            // Calculate cosine similarity using pgvector's distance function
            var similarity = await connection.QuerySingleAsync<double>(@"
                SELECT 1 - (@emb1 <=> @emb2)",
                new { 
                    emb1 = embedList[i].embedding, 
                    emb2 = embedList[j].embedding 
                });
            Console.Write($"{similarity:F3}".PadRight(6));
        }
    }
    Console.WriteLine($" | {embedList[i].text_content.Substring(0, Math.Min(30, embedList[i].text_content.Length))}...");
}

## Cleanup & Summary

Clean up resources and summarize the notebook results.

In [ ]:
// Final summary
Console.WriteLine("🎉 PostgreSQL + pgvector ML Workflow Complete!\n");
Console.WriteLine("What we accomplished:");
Console.WriteLine("✅ Set up ML experiment tracking schema");
Console.WriteLine("✅ Created vector embedding storage with pgvector");
Console.WriteLine("✅ Implemented similarity search functionality");
Console.WriteLine("✅ Analyzed experiment statistics and patterns");
Console.WriteLine("✅ Generated similarity matrix for document clustering");
Console.WriteLine();
Console.WriteLine("🔗 Key Technologies Used:");
Console.WriteLine("  - PostgreSQL with pgvector extension");
Console.WriteLine("  - .NET Interactive with Npgsql driver");
Console.WriteLine("  - Dapper for simplified SQL operations");
Console.WriteLine("  - Vector similarity search with cosine distance");
Console.WriteLine("  - HNSW indexing for performance optimization");

// Optionally clean up test data
Console.WriteLine("\n🧹 To clean up test data, run:");
Console.WriteLine("DROP SCHEMA ml_schema CASCADE;");

connection.Close();